In [ ]:
%pip install openai
%pip install python-dotenv

# AOAI利用準備

In [ ]:
import os
from dotenv import load_dotenv
import openai

load_dotenv()

openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_key = os.getenv("OPENAI_API_KEY")

engine = os.getenv("DEPLOYMENT_NAME")

In [ ]:
engine

In [ ]:
openai.api_version

# 関数定義

## 並列リクエスト関数

In [ ]:
import ray
import json
import openai
import time
import random
import re

from retrying import retry
import multiprocessing

from openai import AzureOpenAI
from dotenv import load_dotenv

import os

@retry(stop_max_attempt_number=10, wait_fixed=3000)
@ray.remote
def unit_AOAI_FC(env_info, index, func_type, tools, prompt, example):
    # clientの作成
    client = AzureOpenAI(
      azure_endpoint=env_info["endpoint"],  
      api_key=env_info["api_key"],  
      api_version = env_info["api_version"]
    )
    # プロンプトの埋込、作成
    content = prompt.format_map(example)
    messages= [
        {"role": "system", "content": content}
    ]
    
    try:
        # APIへのリクエスト
        response = client.chat.completions.create(
            model=env_info["engine"],
            messages=messages,
            tools=tools,
            tool_choice={"type": "function", "function": {"name": f"aug_{func_type}"}}
        )
        
        response_message = response.choices[0].message
        augdata = json.loads(response_message.tool_calls[0].function.arguments)
        
        return 0, augdata[func_type], index, example
    
    except:
        return 1, "ERROR", index, example

def multiRequestProcAOAI_FC(func_type, prompt, tools, examples):
    results = []
    aug_data = []
    index = []
    input_examples = []
    
    # func_name = "aug_" + func_type
    
    # AOAI設定情報のロード
    load_dotenv()
    
    env_info = {
        "endpoint" : os.getenv("OPENAI_API_BASE"),
        "api_key" : os.getenv("OPENAI_API_KEY"),
        "api_version" : os.getenv("OPENAI_API_VERSION"),
        "engine" : os.getenv("DEPLOYMENT_NAME")
    }
    print(env_info["api_version"])
    
    # 並列処理情報定義
    cpu_nums = multiprocessing.cpu_count()
    print(f"Parallel Node:{cpu_nums}")
    ray.init(num_cpus=cpu_nums, ignore_reinit_error=True)
    
    proc_start_time = time.time()
    
    # 分散処理開始
    batch_size = 30
    for i in range(0, len(examples), batch_size):
        batches = examples[i:i + batch_size]
        index_list = [i for i in range(i, i + batch_size)]
        
        start_time = time.time()
        for index, batch in zip(index_list, batches):
            results.append(unit_AOAI_FC.remote(env_info, index, func_type, tools, prompt, batch))
        
        elapsed = time.time() - start_time
        print(f"batch:{i+1}~{i+batch_size} elapsed_time:{elapsed}")
        
        # バッチ毎に60秒間隔を開けてトークンレート制限を回避
        used_time = time.time() - start_time
        
        # 繰り返しのラストはsleepさせない
        if i + batch_size >= len(examples):
            break
        else:
            print(f"sleeptime:{60 - used_time}")
            time.sleep(60 - used_time)
        
    # 結果取得
    results = ray.get(results)

    aug_data = [res[1] for res in results if res[1] is not None]
    index = [res[2] for res in results if res[2] is not None]
    input_examples = [res[3] for res in results if res[3] is not None]

    ray.shutdown()
    
    print("all processed time:", time.time() -  proc_start_time)

    return aug_data, index, input_examples

## PDF分割、テキスト抽出関数

In [ ]:
import os
from PyPDF2 import PdfReader, PdfWriter
from pdfminer.high_level import extract_text

def split_pdf(pdf_filename):
    car_category = pdf_filename.split("_")[0]
    output_folder = f'input/{car_category}'

    # 保存先（車種）のディレクトリがなければ作成
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    pdf = PdfReader(pdf_filename)
    for page in range(len(pdf.pages)):
        pdf_writer = PdfWriter()
        pdf_writer.add_page(pdf.pages[page])

        output = f'{output_folder}/{car_category}_{str(page).zfill(3)}.pdf'
        with open(output, 'wb') as output_pdf:
            pdf_writer.write(output_pdf)

def extract_text_from_pdf(pdf_filename):
    return extract_text(pdf_filename)

## データ作成関数定義

In [ ]:
import glob
import os
import pandas as pd

import re

def make_examples(target_company, aug_q=True, target_qfile=None, key_name=None):
    # Sample Execution Example
    # Q拡張時のサンプル
    # examples = make_examples(target_company="dummy")
    # A拡張時のサンプル
    # examples = make_examples(target_company="dummy", aug_q=False, target_qfile="dummy_questions", key_name="questions")
    
    if aug_q == True:
        print("OK!")
        print("start to augment questions")
        df_qcategory = pd.read_csv("seed_q.csv",sep="\t")
    
    elif aug_q == False:
        if target_qfile is None or key_name is None:
            print("NG!")
            print("Please set target_qfile or key_name or both!!!")
            return None
        else:
            print("OK!")
            print("start to augment answers")
            augedQ_df = pd.read_csv(f"output/{target_qfile}.csv", sep="\t")

    company = target_company

    df_car = pd.read_csv("manual_list.csv", sep="\t")
    df_car_filtered = df_car[df_car["company"] == company]
        

    # ディレクトリのパスを指定（例：'/path/to/directory'）
    directory = f'input/{company}'

    start_page = int(df_car_filtered['startpage'].iloc[0])
    end_page = int(df_car_filtered['endpage'].iloc[0])

    print(start_page)
    print(end_page)

    # 指定されたパターンに一致するファイル名のリストを取得
    file_pattern = os.path.join(directory, f'{company}_*.pdf')
    files = glob.glob(file_pattern)

    examples = []
    
    # 各ファイルに対してループ
    for i, file_path in enumerate(files):
        # ここで各ファイルに対する処理を実行
        if i < start_page or i > end_page:
            continue

        else:
            tmp_dict = {}
            # print(file_path)
            # PDFテキストの取得
            page_detail = extract_text_from_pdf(file_path).replace("\n", "")
            page_detail = re.sub(r'\(cid:\d+\)', '', page_detail)
            
            # Q拡張の場合
            if aug_q == True:
                sample_df = df_qcategory.sample(n=10)
                combined_values = [data["type"] + " | " + data["seed_q"] for i,data in sample_df.iterrows()]
                tmp_dict = {
                    "manual" : page_detail,
                    **{"perspective_"+str(j+1) : combined_values[j] for j in range(10)}
                }
                
            # それ以外の場合    
            else:
                # ページ数とインデックスの差分を吸収
                index = i - start_page
                filtered_df = augedQ_df[augedQ_df['index'] == index]
                
                # ERRORや10件未満のQは無視
                if len(filtered_df) == 10:
                    emb_data = filtered_df[key_name].to_list()
                    tmp_dict = {
                        "manual" : page_detail,
                        **{key_name+"_"+str(j+1) : emb_data[j] for j in range(10)}
                    }
                    
                else:
                    continue
                    
            examples.append(tmp_dict)
    
    return examples
        

# 定数

## リクエスト用定数

In [ ]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "aug_questions",
            "description": "拡張生成した10個のQuestionが入ったリストを保存する",
            "parameters": {
                "type": "object",
                "properties": {
                    "questions": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "配列の要素数は10個であること。出力形式は右記の通り。['que1','que2','que3','que4','que5','que6','que7','que8','que9','que10']",
                    },
                },
            },
            "required": ["questions"],
        },
    },{
        "type": "function",
        "function": {
            "name": "aug_answers",
            "description": "拡張生成した10個のAnswerが入ったリストを保存する",
            "parameters": {
                "type": "object",
                "properties": {
                    "answers": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "配列の要素数は10個であること。出力形式は右記の通り。['ans1','ans2','ans3','ans4','ans5','ans6','ans7','ans8','ans9','ans10']",
                    },
                },
            },
            "required": ["answers"],
        },
    }
]

# 入力データ作成

## データの前処理

### PDF 分割

In [ ]:
split_pdf("dummy.pdf")

# Q拡張

## リクエストデータの整形

In [ ]:
company = "dummy"
examples = make_examples(target_company=company)

In [ ]:
examples[0]

In [ ]:
len(examples)

## リクエスト

### 20240125

In [ ]:
prompt = """【関数呼出】
### ロール
あなたは車のオーナーとしてロールプレイしてください。

### メインタスク
あなたは車に関して気になる質問があり、それらをリストとして保存する必要があります。
与えられたマニュアル「入力」に記載された内容に基づいて、質問を10個日本語で作成してください。

ただし質問作成にあたって絶対に以下の「制約条件」に従いなさい。

### 制約条件
- 使用言語は日本語です。それ以外の言語で出力してはいけません。
- 各質問対象となる装置名や操作方法は原則異なるものを対象にしてください。もし同一のものである場合、文章表現を大きく変えてください。
- 質問は絶対に「入力」から一意的に具体的な回答ができるシンプルなもので、1質問毎に1つだけ質問を生成しなさい。下記の「回答根拠、質問」を参考にしてください。
    例)回答根拠：Aという部品を取り外すときは、まずノブを右に回してください。そのあとに、カチッと音が鳴るまで押し込んで下さい。
       質問　　：["部品Aを外すときにノブを回す方向は？",
       　　　　　 "ノブを回して外す部品は？",
       　　　　　 "Aを外すときに右に回すものは？",
            　　  "どうなるまで押し込めばいい？",
                  "押し込んだら何が鳴る？",
                  "押し込む前に何をする必要がある？",
                  "どうしてノブを右に回すのですか？",
                  "カチッと音が鳴るまで押し込む前にすることは？",
                  "押し込んだらどのような音が鳴る？",
                  ""ノブは右にどうすればいい？,]
            
- 質問の作成において以下の観点に基づいてそれぞれ質問を作成してください。下記のテキストはあくまで一例です。観点（category1:category2）のみ従ってにしてください。
    - {perspective_1}
    - {perspective_2}
    - {perspective_3}
    - {perspective_4}
    - {perspective_5}
    - {perspective_6}
    - {perspective_7}
    - {perspective_8}
    - {perspective_9}
    - {perspective_10}

- 「入力」のマニュアルから回答ができない質問は一切生成しないで下さい。
- 出力フォーマットはリストの要素ごとに簡潔な質問一文とし、不要な情報は一切出力してはいけません。
- 使用言語は日本語です。それ以外の言語で出力してはいけません。

### 入力
{manual}

### 出力
"""

## APIリクエスト並列実行

In [ ]:
target_data = examples

In [ ]:
aug_questions, index, _ = multiRequestProcAOAI_FC("questions", prompt, TOOLS, target_data)

In [ ]:
print(len(aug_questions))
print(len(index))

In [ ]:
aug_questions

In [ ]:
type(index)

In [ ]:
flattened_questions = []
indices = []

# for i,item in enumerate(aug_questions):
#     if type(item) == list:
#         for q in item:
#             flattened_questions.append(q)
#             indices.append(index[i])
#     else:
#         flattened_questions.append(item)
#         indices.append(index[i])

for i,item in enumerate(aug_questions):
    if type(item) == list:
        if len(item) != 10:
            print("not10->", i)
            continue
        
        else:
            for q in item:
                flattened_questions.append(q)
                indices.append(index[i])

In [ ]:
print(len(flattened_questions))
print(len(indices))

In [ ]:
import pandas as pd

df = pd.DataFrame({"questions": flattened_questions, 
                   "index": indices})

# CSVファイルに書き出し
df.to_csv(f'output/{company}_questions.csv', index=False, sep="\t")

# A拡張

## データ整形

In [ ]:
examples = make_examples(target_company=company, aug_q=False, target_qfile=f"{company}_questions", key_name="questions")

In [ ]:
len(examples)

In [ ]:
examples[0]

## リクエスト

In [ ]:
prompt = """【関数呼出】
### ロール
あなたは車のディーラーとしてロールプレイしてください。

### メインタスク
あなたはある車のオーナーから10個の質問を与えられました。それらへの回答を作成し、リストとして保存する必要があります。
与えられた「入力_マニュアル」の内容に基づいて、「入力_質問」への回答を10個それぞれ日本語で作成してください。

ただし、回答生成にあたって絶対に以下の「制約条件」に従いなさい。

### 制約条件
- 出力フォーマットはリストの要素ごとに簡潔な回答一文とし、不要な改行やタブのみや意味のないテキストなど回答に不要な情報は一切出力してはいけません。
- 回答が同一の出力にならないようにして下さい。同じ回答になる場合は、表現を変えてください。
- 使用言語は日本語です。それ以外の言語で出力してはいけません。
- step by stepで考えてください。
    1.「入力_マニュアル」から回答に必要な箇所を抜き出してください。
        - 下記の「回答」を参考にしてください。
        例)マニュアル："Aという部品を取り外すときは、まずノブを右に回してください。そのあとに、カチッと音が鳴るまで押し込んで下さい。",
        例)抜出　　　："まずノブを右に回してください",
        
    2.質問の意図に即して回答を生成して下さい。
        例)質問　　　："部品Aを外すときにノブを回す方向は？",
        例)抜出　　　："まずノブを右に回してください",
        例)回答　　　："右です。
        
- 「入力」のマニュアルから具体的かつ簡潔な回答ができない場合、その旨を伝えてください。その際断りの表現は質問毎に個別に変えてランダムにしてください。


### 入力_マニュアル
{manual}

### 入力_質問
    ["1.{questions_1}",
     "2.{questions_2}",
     "3.{questions_3}",
     "4.{questions_4}",
     "5.{questions_5}",
     "6.{questions_6}",
     "7.{questions_7}",
     "8.{questions_8}",
     "9.{questions_9}",
     "10.{questions_10}"]

### 出力
"""

In [ ]:
target_data = examples

In [ ]:
len(target_data)

In [ ]:
target_data[-1]

In [ ]:
aug_answers, index, input_examples = multiRequestProcAOAI_FC("answers", prompt, TOOLS, target_data)

In [ ]:
len(aug_answers)

In [ ]:
input_examples[1]

In [ ]:
flattened_answers = []
flattened_questions = []
flattened_manuals = []

for i, (a_item, q_item) in enumerate(zip(aug_answers, input_examples)):
    if type(a_item) == list and len(a_item) == 10:
        for item_index, ans in enumerate(a_item):
            q_index = item_index + 1
            
            flattened_questions.append(q_item[f"questions_{q_index}"])
            flattened_manuals.append(q_item["manual"])
            flattened_answers.append(ans)       

In [ ]:
print(len(flattened_answers))
print(len(flattened_questions))
print(len(flattened_manuals))

In [ ]:
i=0

print(flattened_questions[i])
print(flattened_manuals[i])
print(flattened_answers[i])

In [ ]:
import pandas as pd

df = pd.DataFrame(
    {"instruct": flattened_questions,
     "input": flattened_manuals,
     "output": flattened_answers
    }
)

# CSVファイルに書き出し
df.to_csv(f'output/{company}_alldata_pov.csv', index=False, sep="\t")

# LLM学習データ準備

In [ ]:
import pandas as pd

df = pd.read_csv(f"output/{company}_alldata_pov.csv", sep="\t")

In [ ]:
from sklearn.model_selection import train_test_split

train_df, valid_df = train_test_split(df, train_size=2000)

train_df.to_csv('output/train_data.csv', index=False)
valid_df.to_csv('output/valid_data.csv', index=False)

In [ ]:
print(len(train_df))
print(len(valid_df))